# Module 5.4: Real-Time Inference and Alerts - HANDS-ON VERSION

## Combined Case Study: Cybersecurity, Edge AI and Autonomous Driving

---

## 🎯 HANDS-ON LEARNING OBJECTIVES

**⚠️ CODE COMPLETION WORKSHOP:** This notebook provides hands-on practice implementing a complete real-time anomaly detection system. You'll build each component with guided TODO exercises.

**📚 WHAT YOU'LL LEARN:**
- Real-time data streaming simulation techniques
- PyTorch model inference in production environments  
- Alert management systems with cooldown mechanisms
- Performance monitoring and visualization for Edge AI
- End-to-end real-time cybersecurity pipeline development

**🚀 PRACTICAL SKILLS:**
- Implement streaming data processors
- Build inference engines with performance tracking
- Create intelligent alerting systems
- Design monitoring dashboards for live systems
- Handle real-time anomaly detection workflows

---

## Objective

Simulate **real-time inference** on a connected autonomous vehicle using the trained multimodal Edge AI model from Notebook 02.

The simulation will:
- Continuously receive **vehicle telemetry data** and **network traffic data** in a streaming-like fashion
- Predict anomalies in real-time using our PyTorch model
- Trigger different alerts depending on the type of anomaly detected
- Log and visualize alerts for system monitoring

---

## Key Features

### **Real-Time Processing Pipeline:**
```
Streaming Data → Preprocessing → Model Inference → Alert System → Logging
```

### **Alert Types:**
- **Physical Anomaly**: Vehicle sensor/behavior issues
- **Network Anomaly**: Cybersecurity threats
- **Normal Operation**: System functioning correctly

### **System Components:**
1. **Model Loading**: Import trained PyTorch model and preprocessors (SKIP - Step 1)
2. **Data Streaming**: Simulate real-time data ingestion (HANDS-ON - Step 2)
3. **Real-Time Inference**: Continuous prediction pipeline (HANDS-ON - Step 3)
4. **Alert Management**: Smart alerting with cooldown mechanisms (HANDS-ON - Step 4)
5. **Logging & Visualization**: Monitor system performance (HANDS-ON - Step 5-7)

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import warnings
from datetime import datetime, timedelta
from pathlib import Path
import json
from collections import deque, defaultdict

# PyTorch libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler

# Configuration
warnings.filterwarnings('ignore')
np.random.seed(42)
torch.manual_seed(42)
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("Real-Time Inference System - Libraries Loaded!")
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")
print(f"System ready for real-time anomaly detection!")

## Step 1: Model Recriation and weight loading

Create the model architecture and load the pre-trained weights from the last notebook.

In [ ]:
# Recreate the model architecture (must match Notebook 02)
class MultimodalEdgeAI(nn.Module):
    """
    Large MobileNetV2-sized multimodal neural network for enhanced performance
    """
    
    def __init__(self, vehicle_input_size, network_input_size, num_classes=3):
        super(MultimodalEdgeAI, self).__init__()
        
        # Large Vehicle telemetry branch (6 layers)
        self.vehicle_branch = nn.Sequential(
            nn.Linear(vehicle_input_size, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.25),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.15),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.1),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.BatchNorm1d(16)
        )
        
        # Large Network traffic branch (6 layers)
        self.network_branch = nn.Sequential(
            nn.Linear(network_input_size, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.25),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.15),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.1),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.BatchNorm1d(16)
        )
        
        # Attention mechanism
        self.attention = nn.Sequential(
            nn.Linear(32, 64),
            nn.Tanh(),
            nn.Linear(64, 32),
            nn.Softmax(dim=1)
        )
        
        # Large Fusion layers (7 layers)
        self.fusion = nn.Sequential(
            nn.Linear(32, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.4),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.35),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.25),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.2),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.BatchNorm1d(16),
            nn.Dropout(0.1),
            nn.Linear(16, num_classes)
        )
        
    def forward(self, vehicle_input, network_input):
        vehicle_features = self.vehicle_branch(vehicle_input)
        network_features = self.network_branch(network_input)
        fused_features = torch.cat([vehicle_features, network_features], dim=1)
        attention_weights = self.attention(fused_features)
        attended_features = fused_features * attention_weights
        output = self.fusion(attended_features)
        return output

def load_trained_model_and_preprocessors():
    """
    Load the trained model and preprocessing components
    
    Returns:
    - model: Loaded PyTorch model
    - vehicle_scaler: StandardScaler for vehicle features
    - network_scaler: StandardScaler for network features
    - feature_columns: Dictionary with feature column names
    """
    
    print("Loading trained model and preprocessors...")
    
    # Load the dataset to get feature information
    try:
        dataset = pd.read_csv('combined_dataset.csv')
        print(f"Dataset loaded: {dataset.shape}")
        
        # Identify feature columns
        vehicle_features = [col for col in dataset.columns if col.startswith('veh_')]
        network_features = [col for col in dataset.columns if col.startswith('net_')]
        
        print(f"   Vehicle features: {len(vehicle_features)}")
        print(f"   Network features: {len(network_features)}")
        
        # Initialize model with correct input sizes
        model = MultimodalEdgeAI(
            vehicle_input_size=len(vehicle_features),
            network_input_size=len(network_features),
            num_classes=3
        )
        
        # Try to load trained model weights
        try:
            model.load_state_dict(torch.load('best_improved_model.pth', map_location=device))
            print("Trained model weights loaded successfully!")
        except FileNotFoundError:
            print("No trained model found. Using randomly initialized model.")
            print("   Please run Notebook 02 first to train the model.")
        
        model.to(device)
        model.eval()  # Set to evaluation mode
        
        # Create and fit scalers using the full dataset
        vehicle_scaler = StandardScaler()
        network_scaler = StandardScaler()
        
        X_vehicle = dataset[vehicle_features].values
        X_network = dataset[network_features].values
        
        # Handle missing values
        from sklearn.impute import SimpleImputer
        imputer_vehicle = SimpleImputer(strategy='median')
        imputer_network = SimpleImputer(strategy='median')
        
        X_vehicle = imputer_vehicle.fit_transform(X_vehicle)
        X_network = imputer_network.fit_transform(X_network)
        
        # Fit scalers
        vehicle_scaler.fit(X_vehicle)
        network_scaler.fit(X_network)
        
        feature_columns = {
            'vehicle': vehicle_features,
            'network': network_features
        }
        
        print("Preprocessors initialized successfully!")
        
        return model, vehicle_scaler, network_scaler, feature_columns, dataset
        
    except FileNotFoundError:
        print("Dataset not found! Please run Notebook 01 first.")
        raise FileNotFoundError("combined_dataset.csv not found")

# Load model and preprocessors
model, vehicle_scaler, network_scaler, feature_columns, reference_dataset = load_trained_model_and_preprocessors()

# Model information
param_count = sum(p.numel() for p in model.parameters())
model_size_mb = param_count * 4 / (1024 * 1024)

print(f"\nModel Information:")
print(f"   Parameters: {param_count:,}")
print(f"   Size: {model_size_mb:.2f} MB")
print(f"   Vehicle features: {len(feature_columns['vehicle'])}")
print(f"   Network features: {len(feature_columns['network'])}")

## Step 2: Real-Time Data Streaming Simulation - HANDS-ON PRACTICE

**🎯 LEARNING OBJECTIVE:** Learn to build real-time data streaming systems for continuous anomaly detection.

**⚠️ CODE COMPLETION EXERCISE:** You'll implement streaming data simulation, anomaly injection, and batch generation for real-time processing.

In [ ]:
class RealTimeDataStreamer:
    """
    Simulates real-time streaming of vehicle telemetry and network traffic data
    """
    
    def __init__(self, dataset, vehicle_features, network_features, anomaly_rate=0.15):
        # TODO: Initialize data streamer attributes
        # HINT: Store dataset, features, anomaly rate, and current position
        self.dataset = # TODO: Store dataset
        self.vehicle_features = # TODO: Store vehicle feature names
        self.network_features = # TODO: Store network feature names
        self.anomaly_rate = # TODO: Store anomaly injection rate
        self.current_index = 0
        # TODO: Calculate total number of samples
        # HINT: Use len(dataset)
        self.total_samples = # TODO: Get dataset length
        
        # TODO: Create class mapping dictionary
        # HINT: {0: 'Normal', 1: 'Physical Anomaly', 2: 'Network Anomaly'}
        self.class_names = # TODO: Define class name mapping
        
        print(f"Data Streamer initialized:")
        print(f"   Total samples available: {self.total_samples:,}")
        print(f"   Anomaly injection rate: {self.anomaly_rate:.1%}")
    
    def get_next_sample(self, inject_anomaly=None):
        """
        Get the next data sample from the stream
        
        Parameters:
        - inject_anomaly: Force specific anomaly type (0, 1, 2) or None for natural
        
        Returns:
        - sample_data: Dictionary with vehicle/network features and metadata
        """
        
        # TODO: Handle dataset wraparound when reaching the end
        # HINT: Reset current_index to 0 if it reaches total_samples
        if self.current_index >= self.total_samples:
            self.current_index = # TODO: Reset index to beginning
        
        # TODO: Get sample from dataset at current index
        # HINT: Use self.dataset.iloc[self.current_index].copy()
        sample = # TODO: Get current sample from dataset
        # TODO: Increment current index for next call
        self.current_index += 1
        
        # TODO: Extract vehicle and network features from sample
        # HINT: Use sample[self.vehicle_features].values and sample[self.network_features].values
        vehicle_data = # TODO: Extract vehicle features as numpy array
        network_data = # TODO: Extract network features as numpy array
        # TODO: Get true label, default to 0 if not present
        # HINT: Use sample['label'] if 'label' in sample else 0
        true_label = # TODO: Get true label from sample
        
        # TODO: Handle anomaly injection
        if inject_anomaly is not None:
            # TODO: Override true label with injected anomaly type
            true_label = # TODO: Set true_label to inject_anomaly
            if inject_anomaly == 1:  # Physical anomaly
                # TODO: Inject physical anomaly into vehicle data
                # HINT: Call self._inject_physical_anomaly(vehicle_data)
                vehicle_data = # TODO: Inject physical anomaly
            elif inject_anomaly == 2:  # Network anomaly
                # TODO: Inject network anomaly into network data
                # HINT: Call self._inject_network_anomaly(network_data)
                network_data = # TODO: Inject network anomaly
        
        # TODO: Create sample data dictionary
        # HINT: Include timestamp, features, labels, and sample_id
        sample_data = {
            'timestamp': # TODO: Get current timestamp using datetime.now()
            'vehicle_features': # TODO: Add vehicle_data
            'network_features': # TODO: Add network_data
            'true_label': # TODO: Add true_label
            'true_label_name': # TODO: Get class name using self.class_names[true_label]
            'sample_id': # TODO: Add sample ID (current_index - 1)
        }
        
        return sample_data
    
    def _inject_physical_anomaly(self, vehicle_data):
        """Inject physical anomaly into vehicle telemetry"""
        # TODO: Create a copy of vehicle data to avoid modifying original
        # HINT: Use vehicle_data.copy()
        vehicle_data = # TODO: Copy vehicle data
        
        # TODO: Simulate brake system anomaly
        # HINT: Modify first feature to extreme brake pressure (multiply by 2.5, min 100)
        if len(vehicle_data) > 0:
            vehicle_data[0] = # TODO: Set extreme brake pressure
        
        # TODO: Simulate tire pressure anomaly  
        # HINT: Modify fourth feature to low tire pressure (multiply by 0.3, max 10)
        if len(vehicle_data) > 3:
            vehicle_data[3] = # TODO: Set low tire pressure
        
        return vehicle_data
    
    def _inject_network_anomaly(self, network_data):
        """Inject network anomaly into network traffic"""
        # TODO: Create a copy of network data
        network_data = # TODO: Copy network data
        
        # TODO: Simulate suspicious network activity - high packet count
        # HINT: Modify first feature (multiply by 3.0, min 1000)
        if len(network_data) > 0:
            network_data[0] = # TODO: Set high packet count
        
        # TODO: Simulate large payload size
        # HINT: Modify third feature (multiply by 2.0, min 500)
        if len(network_data) > 2:
            network_data[2] = # TODO: Set large payload size
        
        return network_data
    
    def generate_streaming_batch(self, batch_size=10, include_anomalies=True):
        """
        Generate a batch of streaming samples
        
        Parameters:
        - batch_size: Number of samples to generate
        - include_anomalies: Whether to include some anomalies
        
        Returns:
        - List of sample data dictionaries
        """
        # TODO: Initialize empty batch list
        batch = # TODO: Create empty list for batch
        
        for i in range(batch_size):
            # TODO: Determine if we should inject an anomaly
            inject_anomaly = None
            if include_anomalies and # TODO: Check random probability < self.anomaly_rate:
                # TODO: Randomly choose anomaly type (1 or 2)
                # HINT: Use np.random.choice([1, 2]) for Physical or Network anomaly
                inject_anomaly = # TODO: Choose random anomaly type
            
            # TODO: Get next sample with potential anomaly injection
            # HINT: Call self.get_next_sample(inject_anomaly=inject_anomaly)
            sample = # TODO: Get next sample
            # TODO: Add sample to batch
            batch.append(# TODO: Add sample)
            
            # TODO: Add small delay to simulate real-time streaming
            # HINT: Use time.sleep with a small value like 0.001
            # TODO: Add streaming delay
        
        return batch

# TODO: Initialize data streamer
print("Initializing real-time data streamer...")
# TODO: Create RealTimeDataStreamer instance
# HINT: Use reference_dataset, feature_columns['vehicle'], feature_columns['network']
data_streamer = # TODO: Create data streamer instance

# TODO: Test the streamer
print("\\nTesting data streamer:")
# TODO: Generate test batch of 5 samples
# HINT: Call data_streamer.generate_streaming_batch(batch_size=5)
test_batch = # TODO: Generate test batch
for i, sample in enumerate(test_batch):
    print(f"   Sample {i+1}: {sample['true_label_name']} at {sample['timestamp'].strftime('%H:%M:%S')}")

## Step 3: Real-Time Inference Engine - HANDS-ON PRACTICE

**🎯 LEARNING OBJECTIVE:** Learn to build high-performance real-time inference systems for production Edge AI applications.

**⚠️ CODE COMPLETION EXERCISE:** You'll implement preprocessing pipelines, model inference, performance tracking, and prediction result formatting.

In [ ]:
class RealTimeInferenceEngine:
    """
    Real-time inference engine for multimodal anomaly detection
    """
    
    def __init__(self, model, vehicle_scaler, network_scaler, confidence_threshold=0.7):
        # TODO: Initialize inference engine attributes
        # HINT: Store model, scalers, threshold, and class names
        self.model = # TODO: Store model
        self.vehicle_scaler = # TODO: Store vehicle scaler
        self.network_scaler = # TODO: Store network scaler
        self.confidence_threshold = # TODO: Store confidence threshold
        
        # TODO: Define class name mapping
        # HINT: {0: 'Normal', 1: 'Physical Anomaly', 2: 'Network Anomaly'}
        self.class_names = # TODO: Define class name mapping
        
        # TODO: Initialize performance tracking with deque for fixed-size history
        # HINT: Use deque(maxlen=1000) to track last 1000 inference times and predictions
        self.inference_times = # TODO: Create deque for inference times
        self.predictions_history = # TODO: Create deque for predictions history
        
        print(f"Inference Engine initialized:")
        print(f"   Model: {sum(p.numel() for p in model.parameters()):,} parameters")
        print(f"   Confidence threshold: {confidence_threshold}")
        print(f"   Device: {device}")
    
    def preprocess_sample(self, vehicle_features, network_features):
        """
        Preprocess raw features for model input
        
        Parameters:
        - vehicle_features: Raw vehicle telemetry array
        - network_features: Raw network traffic array
        
        Returns:
        - Preprocessed tensors ready for model input
        """
        
        # TODO: Handle missing values by replacing NaN with median
        # HINT: Use np.nan_to_num(array, nan=np.nanmedian(array))
        vehicle_features = # TODO: Replace NaN values in vehicle features
        network_features = # TODO: Replace NaN values in network features
        
        # TODO: Reshape features for scaler (expects 2D array)
        # HINT: Use .reshape(1, -1) to convert 1D to 2D array
        vehicle_scaled = # TODO: Transform and reshape vehicle features
        network_scaled = # TODO: Transform and reshape network features
        
        # TODO: Convert to PyTorch tensors and move to device
        # HINT: Use torch.FloatTensor(scaled_data).to(device)
        vehicle_tensor = # TODO: Convert vehicle data to tensor
        network_tensor = # TODO: Convert network data to tensor
        
        return vehicle_tensor, network_tensor
    
    def run_inference(self, sample_data):
        """
        Run inference on a single sample
        
        Parameters:
        - sample_data: Dictionary containing sample information
        
        Returns:
        - prediction_result: Dictionary with prediction details
        """
        
        # TODO: Record start time for performance measurement
        start_time = # TODO: Get current time
        
        # TODO: Preprocess features for model input
        # HINT: Call self.preprocess_sample with vehicle and network features from sample_data
        vehicle_tensor, network_tensor = # TODO: Preprocess sample features
        
        # TODO: Run model inference without gradient computation
        # HINT: Use torch.no_grad() context manager
        with # TODO: Create no_grad context:
            # TODO: Get model outputs
            # HINT: Call self.model(vehicle_tensor, network_tensor)
            outputs = # TODO: Get model predictions
            
            # TODO: Calculate class probabilities
            # HINT: Use F.softmax(outputs, dim=1)
            probabilities = # TODO: Calculate softmax probabilities
            
            # TODO: Get predicted class index
            # HINT: Use torch.argmax(outputs, dim=1).item()
            predicted_class = # TODO: Get predicted class
            
            # TODO: Get confidence score (maximum probability)
            # HINT: Use torch.max(probabilities).item()
            confidence = # TODO: Get confidence score
        
        # TODO: Calculate inference time in milliseconds
        # HINT: (time.time() - start_time) * 1000
        inference_time_ms = # TODO: Calculate inference time
        # TODO: Store inference time in history
        self.inference_times.append(# TODO: Add inference time)
        
        # TODO: Create comprehensive prediction result dictionary
        # HINT: Include all relevant information from sample_data and inference
        prediction_result = {
            'timestamp': # TODO: Get timestamp from sample_data
            'sample_id': # TODO: Get sample_id from sample_data
            'predicted_class': # TODO: Add predicted_class
            'predicted_label': # TODO: Get class name using self.class_names
            'confidence': # TODO: Add confidence score
            'probabilities': # TODO: Convert probabilities to numpy (use .cpu().numpy()[0])
            'true_class': # TODO: Get true_label from sample_data
            'true_label': # TODO: Get true_label_name from sample_data
            'inference_time_ms': # TODO: Add inference_time_ms
            'is_high_confidence': # TODO: Check if confidence >= self.confidence_threshold
            'is_correct': # TODO: Check if predicted_class == sample_data['true_label']
        }
        
        # TODO: Store prediction in history
        self.predictions_history.append(# TODO: Add prediction_result)
        
        return prediction_result
    
    def get_performance_stats(self):
        """Get current performance statistics"""
        # TODO: Check if we have any inference times recorded
        if not self.inference_times:
            return None
        
        # TODO: Get recent predictions (last 100)
        # HINT: Use list(self.predictions_history)[-100:]
        recent_predictions = # TODO: Get last 100 predictions
        
        # TODO: Calculate performance statistics
        # HINT: Use numpy functions for mean, min, max calculations
        stats = {
            'avg_inference_time_ms': # TODO: Calculate average inference time
            'min_inference_time_ms': # TODO: Calculate minimum inference time
            'max_inference_time_ms': # TODO: Calculate maximum inference time
            'total_predictions': # TODO: Get total number of predictions
            'recent_accuracy': # TODO: Calculate accuracy of recent predictions (mean of 'is_correct')
            'high_confidence_rate': # TODO: Calculate rate of high confidence predictions
        }
        
        return stats

# TODO: Initialize inference engine
# HINT: Use RealTimeInferenceEngine with model, scalers, and confidence threshold
inference_engine = # TODO: Create inference engine

# TODO: Test inference engine
print("\\nTesting inference engine:")
# TODO: Get test sample from data streamer
test_sample = # TODO: Get next sample from data_streamer
# TODO: Run inference on test sample
test_result = # TODO: Run inference

print(f"   Sample processed in {test_result['inference_time_ms']:.2f}ms")
print(f"   Prediction: {test_result['predicted_label']} (confidence: {test_result['confidence']:.3f})")
print(f"   True label: {test_result['true_label']}")
print(f"   Correct: {test_result['is_correct']}")

## Step 4: Smart Alert System - HANDS-ON PRACTICE

**🎯 LEARNING OBJECTIVE:** Learn to design intelligent alerting systems with cooldown mechanisms, priority levels, and comprehensive logging.

**⚠️ CODE COMPLETION EXERCISE:** You'll implement alert triggering logic, cooldown management, priority handling, and alert history tracking.

In [ ]:
class SmartAlertSystem:
    """
    Intelligent alert system with cooldown, priority management, and logging
    """
    
    def __init__(self, cooldown_seconds=10, min_confidence=0.7):
        # TODO: Initialize alert system parameters
        self.cooldown_seconds = # TODO: Store cooldown period
        self.min_confidence = # TODO: Store minimum confidence threshold
        
        # TODO: Initialize alert tracking structures
        # HINT: Use list for history, defaultdict for timestamps and counts
        self.alert_history = # TODO: Create empty list for alert history
        self.last_alert_time = # TODO: Create defaultdict with datetime.min default
        self.alert_counts = # TODO: Create defaultdict with int default (0)
        
        # TODO: Define alert configuration for each class
        # HINT: Dictionary with class IDs as keys, containing emoji, message, priority, color
        self.alert_config = {
            0: {  # Normal
                'emoji': # TODO: Add appropriate emoji for normal operation
                'message': # TODO: Add message for normal operation
                'priority': # TODO: Set priority level for normal (INFO)
                'color': # TODO: Set color code for normal (green: '\033[92m')
            },
            1: {  # Physical Anomaly
                'emoji': # TODO: Add emoji for physical anomaly (🚗)
                'message': # TODO: Add message for vehicle system anomaly
                'priority': # TODO: Set priority level (CRITICAL)
                'color': # TODO: Set color code (red: '\033[91m')
            },
            2: {  # Network Anomaly
                'emoji': # TODO: Add emoji for network anomaly (🔐)
                'message': # TODO: Add message for network security threat
                'priority': # TODO: Set priority level (HIGH)
                'color': # TODO: Set color code (yellow: '\033[93m')
            }
        }
        
        print(f"Smart Alert System initialized:")
        print(f"   Cooldown period: {cooldown_seconds} seconds")
        print(f"   Minimum confidence: {min_confidence}")
    
    def should_trigger_alert(self, prediction_result):
        """
        Determine if an alert should be triggered based on prediction and cooldown
        
        Parameters:
        - prediction_result: Dictionary from inference engine
        
        Returns:
        - should_alert: Boolean indicating if alert should be triggered
        """
        
        # TODO: Extract key information from prediction result
        predicted_class = # TODO: Get predicted_class from prediction_result
        confidence = # TODO: Get confidence from prediction_result
        current_time = # TODO: Get timestamp from prediction_result
        
        # TODO: Check confidence threshold
        # HINT: Return False if confidence < self.min_confidence
        if confidence < # TODO: Check confidence threshold:
            return # TODO: Return False if confidence too low
        
        # TODO: Check cooldown for this alert type
        # HINT: Get last alert time for this class and calculate time difference
        last_alert = # TODO: Get last alert time for this predicted_class
        # TODO: Calculate time since last alert in seconds
        # HINT: (current_time - last_alert).total_seconds()
        time_since_last = # TODO: Calculate time difference
        
        # TODO: Alert logic for anomalies (classes 1 and 2)
        # HINT: Always alert for anomalies if cooldown has passed
        if predicted_class != 0 and time_since_last >= # TODO: Check cooldown period:
            return # TODO: Return True for anomalies after cooldown
        
        # TODO: Alert logic for normal operation (class 0)
        # HINT: Only alert occasionally for normal operation (longer cooldown)
        if predicted_class == 0 and time_since_last >= # TODO: Check extended cooldown (cooldown * 10):
            return # TODO: Return True for normal operation after extended cooldown
        
        return False
    
    def trigger_alert(self, prediction_result):
        """
        Trigger an alert and log it
        
        Parameters:
        - prediction_result: Dictionary from inference engine
        
        Returns:
        - alert_data: Dictionary with alert information
        """
        
        # TODO: Extract prediction details
        predicted_class = # TODO: Get predicted_class
        # TODO: Get configuration for this alert type
        # HINT: Use self.alert_config[predicted_class]
        config = # TODO: Get alert configuration
        timestamp = # TODO: Get timestamp from prediction_result
        
        # TODO: Create comprehensive alert data dictionary
        # HINT: Include all relevant information for logging and display
        alert_data = {
            'timestamp': # TODO: Add timestamp
            'alert_id': # TODO: Create unique alert ID (len(self.alert_history) + 1)
            'type': # TODO: Add predicted_class
            'label': # TODO: Get predicted_label from prediction_result
            'confidence': # TODO: Get confidence from prediction_result
            'priority': # TODO: Get priority from config
            'message': # TODO: Get message from config
            'emoji': # TODO: Get emoji from config
            'sample_id': # TODO: Get sample_id from prediction_result
            'inference_time_ms': # TODO: Get inference_time_ms from prediction_result
            'is_correct': # TODO: Get is_correct from prediction_result
        }
        
        # TODO: Update alert tracking
        self.last_alert_time[predicted_class] = # TODO: Update last alert time
        self.alert_counts[predicted_class] += 1
        self.alert_history.append(# TODO: Add alert_data to history)
        
        # TODO: Display alert to console
        # HINT: Call self._display_alert(alert_data, config)
        self._display_alert(# TODO: Display alert)
        
        return alert_data
    
    def _display_alert(self, alert_data, config):
        """Display alert to console with formatting"""
        
        # TODO: Get display colors
        color = # TODO: Get color from config
        reset_color = # TODO: Set reset color code ('\033[0m')
        
        # TODO: Display formatted alert information
        # HINT: Use f-strings with color codes for formatting
        print(f"\\n{color}{'='*60}")
        print(f"{alert_data['emoji']} {alert_data['priority']} ALERT #{alert_data['alert_id']}")
        print(f"Time: {alert_data['timestamp'].strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Type: {alert_data['label']}")
        print(f"Confidence: {alert_data['confidence']:.3f}")
        print(f"Message: {alert_data['message']}")
        # TODO: Show additional details for anomalies only
        if alert_data['type'] != 0:  # Only show details for anomalies
            print(f"Sample ID: {alert_data['sample_id']}")
            print(f"Processing Time: {alert_data['inference_time_ms']:.2f}ms")
        print(f"{'='*60}{reset_color}")
    
    def get_alert_summary(self):
        """Get summary of all alerts"""
        
        # TODO: Check if we have any alerts
        if not self.alert_history:
            return # TODO: Return appropriate message for no alerts
        
        # TODO: Calculate alert statistics
        total_alerts = # TODO: Get total number of alerts
        # TODO: Get recent alerts (last 5 minutes)
        # HINT: Filter alerts where (datetime.now() - alert['timestamp']).total_seconds() < 300
        recent_alerts = # TODO: Filter recent alerts
        
        # TODO: Create summary dictionary
        summary = {
            'total_alerts': # TODO: Add total_alerts
            'recent_alerts': # TODO: Add count of recent alerts
            'alert_counts': # TODO: Convert self.alert_counts to dict
            'latest_alert': # TODO: Get latest alert or None
        }
        
        return summary
    
    def save_alerts_log(self, filename='alerts_log.csv'):
        """Save alerts to CSV file"""
        
        # TODO: Check if we have alerts to save
        if not self.alert_history:
            print("No alerts to save.")
            return
        
        # TODO: Convert alerts to DataFrame
        # HINT: Use pd.DataFrame(self.alert_history)
        df_alerts = # TODO: Create DataFrame from alert history
        # TODO: Add formatted timestamp column
        # HINT: Use dt.strftime('%Y-%m-%d %H:%M:%S')
        df_alerts['timestamp_str'] = # TODO: Format timestamps
        
        # TODO: Save to CSV file
        # HINT: Use df_alerts.to_csv(filename, index=False)
        # TODO: Save DataFrame to CSV
        print(f"Alerts saved to {filename} ({len(df_alerts)} alerts)")
        
        return df_alerts

# TODO: Initialize alert system
# HINT: Use SmartAlertSystem with cooldown_seconds and min_confidence parameters
alert_system = # TODO: Create alert system

# TODO: Test alert system
print("\\nTesting alert system:")
# TODO: Create test prediction with injected anomaly
test_prediction = # TODO: Run inference with injected anomaly
# TODO: Check if alert should be triggered
if alert_system.should_trigger_alert(# TODO: Pass test_prediction):
    # TODO: Trigger the alert
    test_alert = # TODO: Trigger alert
    print(f"Alert system test completed")
else:
    print("Alert suppressed (cooldown or confidence)")

## Step 5: Real-Time Monitoring Dashboard - HANDS-ON PRACTICE

**🎯 LEARNING OBJECTIVE:** Learn to build comprehensive monitoring systems for production Edge AI applications with metrics tracking and visualization.

**⚠️ CODE COMPLETION EXERCISE:** You'll implement metrics collection, performance tracking, status displays, and monitoring dashboards.

In [ ]:
class RealTimeMonitor:
    """
    Real-time monitoring dashboard for the inference system
    """
    
    def __init__(self, update_interval=10):
        # TODO: Initialize monitoring parameters
        self.update_interval = # TODO: Store update interval
        # TODO: Record start time for runtime calculations
        # HINT: Use datetime.now()
        self.start_time = # TODO: Record start time
        self.monitoring_active = False
        
        # TODO: Initialize metrics tracking
        # HINT: Use empty list to store metrics history
        self.metrics_history = # TODO: Create metrics history list
        
        print(f"Real-Time Monitor initialized")
        print(f"   Update interval: {update_interval} seconds")
    
    def log_metrics(self, inference_engine, alert_system, data_streamer):
        """Log current system metrics"""
        
        # TODO: Get current time and calculate runtime
        current_time = # TODO: Get current datetime
        # TODO: Calculate runtime in seconds
        # HINT: (current_time - self.start_time).total_seconds()
        runtime = # TODO: Calculate runtime
        
        # TODO: Get performance stats from inference engine
        # HINT: Call inference_engine.get_performance_stats()
        perf_stats = # TODO: Get performance statistics
        
        # TODO: Get alert summary from alert system
        # HINT: Call alert_system.get_alert_summary()
        alert_summary = # TODO: Get alert summary
        
        # TODO: Create comprehensive metrics dictionary
        # HINT: Handle case where perf_stats might be None
        metrics = {
            'timestamp': # TODO: Add current_time
            'runtime_seconds': # TODO: Add runtime
            'total_predictions': # TODO: Get total predictions (or 0 if perf_stats is None)
            'avg_inference_time_ms': # TODO: Get average inference time (or 0.0)
            'recent_accuracy': # TODO: Get recent accuracy (or 0.0)
            'high_confidence_rate': # TODO: Get high confidence rate (or 0.0)
            'total_alerts': # TODO: Get total alerts (handle dict or string cases)
            'recent_alerts': # TODO: Get recent alerts (handle dict or string cases)
        }
        
        # TODO: Store metrics in history
        self.metrics_history.append(# TODO: Add metrics)
        
        return metrics
    
    def display_status(self, metrics):
        """Display current system status"""
        
        # TODO: Display real-time status update
        print(f"\\n{'='*50}")
        print(f"📊 REAL-TIME SYSTEM STATUS")
        print(f"{'='*50}")
        print(f"⏱️  Runtime: {metrics['runtime_seconds']:.1f} seconds")
        print(f"📈 Total Predictions: {metrics['total_predictions']:,}")
        
        # TODO: Display performance metrics
        # HINT: Show inference time, accuracy, and confidence rate
        if metrics['total_predictions'] > 0:
            print(f"⚡ Avg Inference Time: {metrics['avg_inference_time_ms']:.2f}ms")
            print(f"🎯 Recent Accuracy: {metrics['recent_accuracy']:.1%}")
            print(f"🔒 High Confidence Rate: {metrics['high_confidence_rate']:.1%}")
        
        # TODO: Display alert information
        # HINT: Show total and recent alerts
        print(f"🚨 Total Alerts: {metrics['total_alerts']}")
        print(f"⚠️  Recent Alerts: {metrics['recent_alerts']}")
        print(f"{'='*50}")
    
    def plot_performance_metrics(self):
        """Plot system performance over time"""
        
        # TODO: Check if we have enough data for plotting
        if len(self.metrics_history) < 2:
            print("Not enough data for plotting")
            return
        
        # TODO: Convert metrics to DataFrame
        # HINT: Use pd.DataFrame(self.metrics_history)
        df_metrics = # TODO: Create DataFrame from metrics history
        
        # TODO: Create subplot figure
        # HINT: Use plt.subplots(2, 2, figsize=(15, 10))
        fig, axes = # TODO: Create subplots
        fig.suptitle('Real-Time System Performance Metrics', fontsize=16, fontweight='bold')
        
        # TODO: Plot 1 - Inference time
        # HINT: Plot avg_inference_time_ms vs index
        axes[0, 0].plot(# TODO: Plot inference time)
        axes[0, 0].set_title('Average Inference Time')
        axes[0, 0].set_ylabel('Time (ms)')
        axes[0, 0].grid(True, alpha=0.3)
        
        # TODO: Plot 2 - Accuracy
        # HINT: Plot recent_accuracy * 100 vs index, set ylim(0, 100)
        axes[0, 1].plot(# TODO: Plot accuracy percentage)
        axes[0, 1].set_title('Recent Accuracy')
        axes[0, 1].set_ylabel('Accuracy (%)')
        # TODO: Set y-axis limits to 0-100%
        axes[0, 1].grid(True, alpha=0.3)
        
        # TODO: Plot 3 - Predictions per update
        # HINT: Calculate difference in total_predictions between updates
        pred_diff = # TODO: Calculate prediction differences
        axes[1, 0].plot(# TODO: Plot prediction rate)
        axes[1, 0].set_title('Predictions per Update')
        axes[1, 0].set_ylabel('Count')
        axes[1, 0].grid(True, alpha=0.3)
        
        # TODO: Plot 4 - Alert rate
        # HINT: Calculate difference in total_alerts between updates
        alert_diff = # TODO: Calculate alert differences
        axes[1, 1].plot(# TODO: Plot alert rate)
        axes[1, 1].set_title('Alerts per Update')
        axes[1, 1].set_ylabel('Count')
        axes[1, 1].grid(True, alpha=0.3)
        
        # TODO: Apply layout and show plot
        # HINT: Use plt.tight_layout() and plt.show()
        # TODO: Apply tight layout
        # TODO: Show the plot

# TODO: Initialize monitor
# HINT: Use RealTimeMonitor with update_interval parameter
monitor = # TODO: Create monitor instance

# TODO: Test monitoring
print("\\nTesting monitoring system:")
# TODO: Log test metrics
test_metrics = # TODO: Log metrics from all components
# TODO: Display status
monitor.display_status(# TODO: Pass test_metrics)

In [ ]:
## Step 6: Real-Time Simulation Execution - HANDS-ON PRACTICE

**🎯 LEARNING OBJECTIVE:** Learn to build complete real-time simulation systems with dynamic scenarios, performance monitoring, and comprehensive result analysis.

**⚠️ CODE COMPLETION EXERCISE:** You'll implement the main simulation loop, timing control, scenario management, and result collection for production Edge AI systems.

def run_realtime_simulation(duration_seconds=60, samples_per_second=2, enable_anomalies=True):
    """
    Run complete real-time simulation with enhanced scenario management
    
    Parameters:
    - duration_seconds: How long to run the simulation
    - samples_per_second: Sampling rate for data processing
    - enable_anomalies: Whether to inject anomalies for testing
    """
    
    print(f"Starting Real-Time Anomaly Detection Simulation")
    print(f"{'='*60}")
    print(f"Duration: {duration_seconds} seconds")
    print(f"Sampling rate: {samples_per_second} samples/second")
    print(f"Anomaly injection: {'Enabled' if enable_anomalies else 'Disabled'}")
    print(f"{'='*60}")
    
    # TODO: Reset systems for fresh simulation
    # HINT: Clear history, counts, and timestamps from alert_system
    alert_system.alert_history.clear()
    alert_system.alert_counts.clear()
    alert_system.last_alert_time.clear()
    
    # Temporarily reduce cooldown for more alerts
    original_cooldown = alert_system.cooldown_seconds
    alert_system.cooldown_seconds = 2  # Reduced cooldown for demo
    
    # TODO: Calculate simulation parameters
    # HINT: Calculate sample_interval from samples_per_second (1.0 / samples_per_second)
    sample_interval = # TODO: Calculate time between samples
    # TODO: Calculate total number of samples
    # HINT: int(duration_seconds * samples_per_second)
    total_samples = # TODO: Calculate total samples needed
    # TODO: Set monitoring update interval
    # HINT: max(8, duration_seconds // 8) for frequent monitoring
    monitor_interval = # TODO: Calculate monitor update frequency
    
    # Define alert scenarios throughout the simulation
    scenario_timeline = {
        # Early phase: Normal operation with occasional anomalies
        (0, 0.2): {
            'name': 'Normal Operations',
            'anomaly_rate': 0.15,
            'physical_weight': 0.4,
            'network_weight': 0.6,
            'burst_probability': 0.05
        },
        # Mid-early: Coordinated attack simulation
        (0.2, 0.35): {
            'name': 'Coordinated Network Attack',
            'anomaly_rate': 0.7,
            'physical_weight': 0.2,
            'network_weight': 0.8,
            'burst_probability': 0.3
        },
        # Mid: System recovery and mixed threats
        (0.35, 0.5): {
            'name': 'Mixed Threat Environment',
            'anomaly_rate': 0.45,
            'physical_weight': 0.6,
            'network_weight': 0.4,
            'burst_probability': 0.15
        },
        # Mid-late: Physical system stress test
        (0.5, 0.7): {
            'name': 'Vehicle System Stress Test',
            'anomaly_rate': 0.6,
            'physical_weight': 0.8,
            'network_weight': 0.2,
            'burst_probability': 0.25
        },
        # Final: System degradation simulation
        (0.7, 1.0): {
            'name': 'System Performance Degradation',
            'anomaly_rate': 0.3,
            'physical_weight': 0.5,
            'network_weight': 0.5,
            'burst_probability': 0.1,
            'inject_performance_issues': True
        }
    }
    
    # TODO: Initialize simulation timing
    start_time = # TODO: Record simulation start time
    last_monitor_time = # TODO: Initialize last monitor update time
    performance_degradation_active = False
    burst_mode_samples = 0
    
    print(f"Processing with dynamic scenario changes...")
    print(f"Monitor updates every {monitor_interval} seconds")
    print(f"\\nSimulation starting...")
    
    try:
        for sample_num in range(total_samples):
            # TODO: Record loop start time for timing control
            loop_start = # TODO: Get loop start time
            
            # TODO: Calculate progress ratio
            # HINT: sample_num / total_samples
            progress_ratio = # TODO: Calculate progress as ratio 0-1
            
            # TODO: Determine current scenario based on progress
            current_scenario = None
            for (start_ratio, end_ratio), scenario in scenario_timeline.items():
                if start_ratio <= progress_ratio < end_ratio:
                    current_scenario = scenario
                    break
            
            # TODO: Handle case where no scenario found (use last scenario)
            if current_scenario is None:
                current_scenario = # TODO: Use last scenario as fallback
            
            # TODO: Implement burst mode logic for consecutive anomalies
            inject_anomaly = None
            if burst_mode_samples > 0:
                # TODO: Continue burst mode
                # HINT: Choose anomaly type based on scenario weights
                inject_anomaly = # TODO: Choose anomaly type during burst
                burst_mode_samples -= 1
            elif np.random.random() < current_scenario.get('burst_probability', 0):
                # TODO: Start new burst (3-7 consecutive anomalies)
                burst_mode_samples = # TODO: Set random burst length (3-8 samples)
                inject_anomaly = # TODO: Choose initial burst anomaly type
            elif np.random.random() < current_scenario['anomaly_rate']:
                # TODO: Regular anomaly injection
                inject_anomaly = # TODO: Choose regular anomaly type
            
            # TODO: Get sample and run inference
            # HINT: Use data_streamer.get_next_sample and inference_engine.run_inference
            sample_data = # TODO: Get next sample with potential anomaly injection
            prediction_result = # TODO: Run inference on sample
            
            # TODO: Simulate performance degradation in final phase
            if current_scenario.get('inject_performance_issues', False):
                # TODO: Randomly degrade inference time
                if np.random.random() < 0.2:
                    prediction_result['inference_time_ms'] *= # TODO: Multiply by random factor (2-4x)
                
                # TODO: Occasionally flip predictions to simulate model degradation
                if np.random.random() < 0.12:
                    original_class = prediction_result['predicted_class']
                    # TODO: Choose alternative class
                    alternatives = # TODO: Create list of other classes
                    new_class = # TODO: Choose random alternative class
                    # TODO: Update prediction result with degraded prediction
                    prediction_result['predicted_class'] = new_class
                    prediction_result['predicted_label'] = inference_engine.class_names[new_class]
                    prediction_result['confidence'] *= # TODO: Reduce confidence (0.4-0.8x)
                    prediction_result['is_correct'] = # TODO: Recalculate correctness
            
            # TODO: Enhanced alert triggering with performance monitoring
            should_alert = # TODO: Check if standard alert should trigger
            
            # TODO: Additional alert conditions for performance and confidence
            if not should_alert:
                # TODO: Alert on high inference time (performance issue)
                if prediction_result['inference_time_ms'] > 50:  # Threshold for concern
                    should_alert = # TODO: Set alert flag for performance issue
                    prediction_result['performance_alert'] = True
                
                # TODO: Alert on confidence drops for anomalies
                elif prediction_result['confidence'] < 0.4 and prediction_result['predicted_class'] != 0:
                    should_alert = # TODO: Set alert flag for low confidence
                    prediction_result['low_confidence_alert'] = True
            
            # TODO: Trigger alerts with enhanced metadata
            if should_alert:
                alert_data = # TODO: Trigger alert
                
                # TODO: Add scenario context to alert
                alert_data['scenario'] = # TODO: Add current scenario name
                if hasattr(prediction_result, 'performance_alert'):
                    alert_data['alert_subtype'] = 'Performance Issue'
                elif hasattr(prediction_result, 'low_confidence_alert'):
                    alert_data['alert_subtype'] = 'Low Confidence'
                
                # TODO: Special handling for burst alerts
                if burst_mode_samples > 0:
                    alert_data['burst_alert'] = True
                    alert_data['remaining_burst'] = burst_mode_samples
            
            # TODO: Monitor updates with scenario info
            current_time = # TODO: Get current time
            if current_time - last_monitor_time >= monitor_interval:
                # TODO: Log metrics and display status
                metrics = # TODO: Log system metrics
                # TODO: Display current status
                
                # TODO: Display current scenario information
                print(f"Current Scenario: {current_scenario['name']}")
                if burst_mode_samples > 0:
                    print(f"🔥 BURST MODE ACTIVE: {burst_mode_samples} samples remaining")
                print(f"Anomaly Rate: {current_scenario['anomaly_rate']:.0%}")
                
                last_monitor_time = current_time
            
            # TODO: Maintain precise timing
            # HINT: Calculate sleep time to maintain sample_interval
            elapsed = # TODO: Calculate loop elapsed time
            sleep_time = # TODO: Calculate required sleep time
            if sleep_time > 0:
                # TODO: Sleep to maintain timing
                pass
            
            # TODO: Progress indicator with scenario transitions
            if (sample_num + 1) % max(1, (total_samples // 15)) == 0:
                progress = (sample_num + 1) / total_samples * 100
                elapsed_sim_time = # TODO: Calculate total simulation elapsed time
                print(f"Progress: {progress:.0f}% ({elapsed_sim_time:.1f}s) - {current_scenario['name']}")

    except KeyboardInterrupt:
        print(f"\\n🛑 Simulation interrupted by user")
    
    finally:
        # TODO: Restore original cooldown
        alert_system.cooldown_seconds = # TODO: Restore original cooldown
    
    # TODO: Calculate final comprehensive statistics
    total_runtime = # TODO: Calculate total simulation runtime
    final_metrics = # TODO: Log final metrics
    
    print(f"\\n🎯 Comprehensive Simulation Complete!")
    print(f"{'='*70}")
    print(f"Total runtime: {total_runtime:.2f} seconds")
    print(f"Samples processed: {final_metrics['total_predictions']}")
    print(f"Processing rate: {final_metrics['total_predictions']/total_runtime:.1f} samples/sec")
    print(f"Average inference: {final_metrics['avg_inference_time_ms']:.2f}ms")
    print(f"Recent accuracy: {final_metrics['recent_accuracy']:.1%}")
    
    # TODO: Detailed alert analysis
    alert_summary = # TODO: Get comprehensive alert summary
    if isinstance(alert_summary, dict):
        print(f"\\n📊 Comprehensive Alert Analysis:")
        print(f"   🚨 Total alerts generated: {alert_summary['total_alerts']}")
        print(f"   ⏰ Recent alerts (5 min): {alert_summary['recent_alerts']}")
        
        # TODO: Alert breakdown by type
        class_names = {0: 'Normal Operation', 1: 'Physical Anomaly', 2: 'Network Anomaly'}
        print(f"\\n   Alert Distribution:")
        for alert_type, count in alert_summary['alert_counts'].items():
            percentage = (count / alert_summary['total_alerts']) * 100 if alert_summary['total_alerts'] > 0 else 0
            print(f"     {class_names.get(alert_type, f'Type {alert_type}')}: {count} alerts ({percentage:.1f}%)")
        
        # TODO: Calculate and display alert rate
        if alert_summary['total_alerts'] > 0:
            alert_rate = # TODO: Calculate alerts per minute
            print(f"\\n   📈 Alert Rate: {alert_rate:.1f} alerts/minute")
    
    # TODO: Save comprehensive results
    alerts_df = # TODO: Save alerts to file
    
    # TODO: Enhanced performance plotting
    if len(monitor.metrics_history) > 1:
        # TODO: Plot performance metrics
        pass
    
    # TODO: Alert timeline visualization
    if len(alert_system.alert_history) > 0:
        # TODO: Plot alert timeline (will be implemented in Step 7)
        pass
    
    return final_metrics, alerts_df

print("✅ Comprehensive Alert Simulation Function Defined")
print("   Ready for multi-scenario simulation with dynamic timeline visualization")

# TODO: Execute the simulation
print("Ready to start real-time simulation!")
print("   The simulation will process streaming data and detect anomalies in real-time.")
print("   Press Ctrl+C to stop the simulation early.")
print("\\n" + "="*60)

# TODO: Run simulation with specified parameters
# HINT: Call run_realtime_simulation with duration_seconds, samples_per_second, enable_anomalies
simulation_results, alerts_log = # TODO: Execute simulation with 30s duration, 1 sample/sec, anomalies enabled

## Step 7: Alert Analysis and Visualization

Analyze the generated alerts and create visualizations for system performance.

In [ ]:
def run_comprehensive_alert_simulation(duration_seconds=90, samples_per_second=4):
    """
    Comprehensive alert simulation with diverse scenarios and patterns
    
    This enhanced simulation includes:
    - Multiple alert scenarios (burst patterns, gradual escalation, system health)
    - Temporal anomaly patterns (coordinated attacks, system degradation)
    - Performance-based alerts (latency spikes, accuracy drops)
    - Mixed confidence levels for realistic alert variety
    """
    
    print(f"Starting Comprehensive Multi-Scenario Alert Simulation")
    print(f"{'='*70}")
    print(f"Duration: {duration_seconds} seconds")
    print(f"High-frequency sampling: {samples_per_second} samples/second")
    print(f"Expected ~{duration_seconds * samples_per_second} total samples")
    print(f"{'='*70}")

    # Reset all systems
    alert_system.alert_history.clear()
    alert_system.alert_counts.clear()
    alert_system.last_alert_time.clear()
    
    # Temporarily reduce cooldown for more alerts
    original_cooldown = alert_system.cooldown_seconds
    alert_system.cooldown_seconds = 2  # Reduced cooldown for demo
    
    # Enhanced simulation parameters
    sample_interval = 1.0 / samples_per_second
    total_samples = int(duration_seconds * samples_per_second)
    monitor_interval = max(8, duration_seconds // 8)  # Frequent monitoring
    
    # Define alert scenarios throughout the simulation
    scenario_timeline = {
        # Early phase: Normal operation with occasional anomalies
        (0, 0.2): {
            'name': 'Normal Operations',
            'anomaly_rate': 0.15,
            'physical_weight': 0.4,
            'network_weight': 0.6,
            'burst_probability': 0.05
        },
        # Mid-early: Coordinated attack simulation
        (0.2, 0.35): {
            'name': 'Coordinated Network Attack',
            'anomaly_rate': 0.7,
            'physical_weight': 0.2,
            'network_weight': 0.8,
            'burst_probability': 0.3
        },
        # Mid: System recovery and mixed threats
        (0.35, 0.5): {
            'name': 'Mixed Threat Environment',
            'anomaly_rate': 0.45,
            'physical_weight': 0.6,
            'network_weight': 0.4,
            'burst_probability': 0.15
        },
        # Mid-late: Physical system stress test
        (0.5, 0.7): {
            'name': 'Vehicle System Stress Test',
            'anomaly_rate': 0.6,
            'physical_weight': 0.8,
            'network_weight': 0.2,
            'burst_probability': 0.25
        },
        # Final: System degradation simulation
        (0.7, 1.0): {
            'name': 'System Performance Degradation',
            'anomaly_rate': 0.3,
            'physical_weight': 0.5,
            'network_weight': 0.5,
            'burst_probability': 0.1,
            'inject_performance_issues': True
        }
    }
    
    start_time = time.time()
    last_monitor_time = start_time
    performance_degradation_active = False
    burst_mode_samples = 0
    
    print(f"Processing with dynamic scenario changes...")
    print(f"Monitor updates every {monitor_interval} seconds")
    print(f"\nSimulation starting...")
    
    try:
        for sample_num in range(total_samples):
            loop_start = time.time()
            progress_ratio = sample_num / total_samples
            
            # Determine current scenario
            current_scenario = None
            for (start_ratio, end_ratio), scenario in scenario_timeline.items():
                if start_ratio <= progress_ratio < end_ratio:
                    current_scenario = scenario
                    break
            
            if current_scenario is None:
                current_scenario = list(scenario_timeline.values())[-1]  # Use last scenario
            
            # Burst mode logic
            inject_anomaly = None
            if burst_mode_samples > 0:
                # Continue burst
                inject_anomaly = np.random.choice([1, 2], 
                    p=[current_scenario['physical_weight'], current_scenario['network_weight']])
                burst_mode_samples -= 1
            elif np.random.random() < current_scenario.get('burst_probability', 0):
                # Start new burst (3-7 consecutive anomalies)
                burst_mode_samples = np.random.randint(3, 8)
                inject_anomaly = np.random.choice([1, 2], 
                    p=[current_scenario['physical_weight'], current_scenario['network_weight']])
            elif np.random.random() < current_scenario['anomaly_rate']:
                # Regular anomaly injection
                inject_anomaly = np.random.choice([1, 2], 
                    p=[current_scenario['physical_weight'], current_scenario['network_weight']])
            
            # Get sample and run inference
            sample_data = data_streamer.get_next_sample(inject_anomaly=inject_anomaly)
            prediction_result = inference_engine.run_inference(sample_data)
            
            # Simulate performance degradation in final phase
            if current_scenario.get('inject_performance_issues', False):
                # Randomly degrade inference time
                if np.random.random() < 0.2:
                    prediction_result['inference_time_ms'] *= np.random.uniform(2.0, 4.0)
                
                # Occasionally flip predictions to simulate model degradation
                if np.random.random() < 0.12:
                    original_class = prediction_result['predicted_class']
                    alternatives = [c for c in [0, 1, 2] if c != original_class]
                    new_class = np.random.choice(alternatives)
                    prediction_result['predicted_class'] = new_class
                    prediction_result['predicted_label'] = inference_engine.class_names[new_class]
                    prediction_result['confidence'] *= np.random.uniform(0.4, 0.8)  # Lower confidence
                    prediction_result['is_correct'] = (new_class == sample_data['true_label'])
            
            # Enhanced alert triggering with performance monitoring
            should_alert = alert_system.should_trigger_alert(prediction_result)
            
            # Additional alert conditions
            if not should_alert:
                # Alert on high inference time (performance issue)
                if prediction_result['inference_time_ms'] > 50:  # Threshold for concern
                    should_alert = True
                    # Modify alert for performance issue
                    prediction_result['performance_alert'] = True
                
                # Alert on confidence drops
                elif prediction_result['confidence'] < 0.4 and prediction_result['predicted_class'] != 0:
                    should_alert = True
                    prediction_result['low_confidence_alert'] = True
            
            # Trigger alerts
            if should_alert:
                alert_data = alert_system.trigger_alert(prediction_result)
                
                # Add scenario context to alert
                alert_data['scenario'] = current_scenario['name']
                if hasattr(prediction_result, 'performance_alert'):
                    alert_data['alert_subtype'] = 'Performance Issue'
                elif hasattr(prediction_result, 'low_confidence_alert'):
                    alert_data['alert_subtype'] = 'Low Confidence'
                
                # Special handling for burst alerts
                if burst_mode_samples > 0:
                    alert_data['burst_alert'] = True
                    alert_data['remaining_burst'] = burst_mode_samples
            
            # Monitor updates with scenario info
            current_time = time.time()
            if current_time - last_monitor_time >= monitor_interval:
                metrics = monitor.log_metrics(inference_engine, alert_system, data_streamer)
                monitor.display_status(metrics)
                
                # Display current scenario
                print(f"Current Scenario: {current_scenario['name']}")
                if burst_mode_samples > 0:
                    print(f"🔥 BURST MODE ACTIVE: {burst_mode_samples} samples remaining")
                print(f"Anomaly Rate: {current_scenario['anomaly_rate']:.0%}")
                
                last_monitor_time = current_time
            
            # Maintain timing
            elapsed = time.time() - loop_start
            sleep_time = max(0, sample_interval - elapsed)
            if sleep_time > 0:
                time.sleep(sleep_time)
            
            # Progress with scenario transitions
            if (sample_num + 1) % max(1, (total_samples // 15)) == 0:
                progress = (sample_num + 1) / total_samples * 100
                elapsed_sim_time = time.time() - start_time
                print(f"Progress: {progress:.0f}% ({elapsed_sim_time:.1f}s) - {current_scenario['name']}")

    except KeyboardInterrupt:
        print(f"\n🛑 Simulation interrupted by user")
    
    finally:
        # Restore original cooldown
        alert_system.cooldown_seconds = original_cooldown
    
    # Final comprehensive statistics
    total_runtime = time.time() - start_time
    final_metrics = monitor.log_metrics(inference_engine, alert_system, data_streamer)
    
    print(f"\n🎯 Comprehensive Simulation Complete!")
    print(f"{'='*70}")
    print(f"Total runtime: {total_runtime:.2f} seconds")
    print(f"Samples processed: {final_metrics['total_predictions']}")
    print(f"Processing rate: {final_metrics['total_predictions']/total_runtime:.1f} samples/sec")
    print(f"Average inference: {final_metrics['avg_inference_time_ms']:.2f}ms")
    print(f"Recent accuracy: {final_metrics['recent_accuracy']:.1%}")
    
    # Detailed alert analysis
    alert_summary = alert_system.get_alert_summary()
    if isinstance(alert_summary, dict):
        print(f"\n📊 Comprehensive Alert Analysis:")
        print(f"   🚨 Total alerts generated: {alert_summary['total_alerts']}")
        print(f"   ⏰ Recent alerts (5 min): {alert_summary['recent_alerts']}")
        
        # Alert breakdown by type
        class_names = {0: 'Normal Operation', 1: 'Physical Anomaly', 2: 'Network Anomaly'}
        print(f"\n   Alert Distribution:")
        for alert_type, count in alert_summary['alert_counts'].items():
            percentage = (count / alert_summary['total_alerts']) * 100 if alert_summary['total_alerts'] > 0 else 0
            print(f"     {class_names.get(alert_type, f'Type {alert_type}')}: {count} alerts ({percentage:.1f}%)")
        
        # Alert rate analysis
        if alert_summary['total_alerts'] > 0:
            alert_rate = alert_summary['total_alerts'] / (total_runtime / 60)  # alerts per minute
            print(f"\n   📈 Alert Rate: {alert_rate:.1f} alerts/minute")
    
    # Save comprehensive results
    alerts_df = alert_system.save_alerts_log()
    
    # Enhanced performance plotting
    if len(monitor.metrics_history) > 1:
        monitor.plot_performance_metrics()
    
    # Alert timeline visualization
    if len(alert_system.alert_history) > 0:
        plot_alert_timeline()
    
    return final_metrics, alerts_df

print("✅ Comprehensive Alert Simulation Function Defined")
print("   Ready for multi-scenario simulation with dynamic timeline visualization")

In [ ]:
def plot_alert_timeline():
    """Create a timeline visualization of alerts with adaptive time binning and full timeline support"""
    
    if not alert_system.alert_history:
        print("No alerts to plot")
        return
    
    df_alerts = pd.DataFrame(alert_system.alert_history)
    
    # Display alert timeline info
    total_alerts = len(df_alerts)
    duration_seconds = (df_alerts['timestamp'].max() - df_alerts['timestamp'].min()).total_seconds()
    print(f"\n📊 Alert Timeline Analysis:")
    print(f"   Total alerts: {total_alerts}")
    print(f"   Simulation duration: {duration_seconds:.1f} seconds ({duration_seconds/60:.1f} minutes)")
    print(f"   Alert rate: {total_alerts/(duration_seconds/60):.1f} alerts/minute")
    
    # Create timeline plot with improved sizing for full timeline
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 12))  # Wider plot for full timeline
    
    # Alert timeline (top plot) - Fixed to show full timeline
    class_colors = {0: 'green', 1: 'red', 2: 'orange'}
    class_names = {0: 'Normal', 1: 'Physical', 2: 'Network'}
    
    # Calculate time in minutes for better readability
    for alert_type in df_alerts['type'].unique():
        type_alerts = df_alerts[df_alerts['type'] == alert_type]
        times = [(t - df_alerts['timestamp'].min()).total_seconds() / 60 for t in type_alerts['timestamp']]
        
        ax1.scatter(times, [alert_type] * len(times), 
                   c=class_colors[alert_type], label=class_names[alert_type], 
                   alpha=0.7, s=60)
    
    # Set x-axis to show full timeline
    max_time_minutes = duration_seconds / 60
    ax1.set_xlim(0, max_time_minutes)
    ax1.set_xlabel('Time (minutes)')
    ax1.set_ylabel('Alert Type')
    ax1.set_title(f'Alert Timeline - Full {duration_seconds:.1f} Second Simulation')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Set y-axis ticks to show alert type labels
    ax1.set_yticks([0, 1, 2])
    ax1.set_yticklabels(['Normal', 'Physical', 'Network'])
    
    # Alert frequency over time (bottom plot) - Improved binning with full timeline coverage
    total_duration_minutes = duration_seconds / 60
    
    if total_duration_minutes <= 2:
        # For short simulations, use 10-second bins
        bin_size_seconds = 10
        time_unit = "10-sec intervals"
    elif total_duration_minutes <= 5:
        # For medium simulations, use 15-second bins  
        bin_size_seconds = 15
        time_unit = "15-sec intervals"
    else:
        # For long simulations, use 30-second bins
        bin_size_seconds = 30
        time_unit = "30-sec intervals"
    
    # Create time bins covering the full simulation duration
    num_bins = int(np.ceil(duration_seconds / bin_size_seconds))
    df_alerts['time_bin'] = ((df_alerts['timestamp'] - df_alerts['timestamp'].min()).dt.total_seconds() / bin_size_seconds).astype(int)
    
    # Count alerts per bin
    alerts_per_bin = df_alerts.groupby('time_bin').size()
    
    # Create complete range of bins (including empty ones) for full timeline
    all_bins = range(0, num_bins + 1)
    alerts_counts = [alerts_per_bin.get(i, 0) for i in all_bins]
    
    # Create bar chart with full timeline
    ax2.bar(all_bins, alerts_counts, alpha=0.7, color='skyblue', edgecolor='navy', linewidth=0.5)
    ax2.set_xlabel(f'Time ({time_unit})')
    ax2.set_ylabel('Alerts per Interval')
    ax2.set_title(f'Alert Frequency Distribution - Full Timeline ({time_unit})')
    ax2.grid(True, alpha=0.3)
    
    # Set x-axis labels for better readability (show key time points)
    if len(all_bins) <= 20:  # Show all labels if not too crowded
        tick_step = max(1, len(all_bins) // 10)
        ax2.set_xticks(all_bins[::tick_step])
        ax2.set_xticklabels([f"{i*bin_size_seconds}s" for i in all_bins[::tick_step]], rotation=45)
    else:  # Show fewer labels for very long simulations
        tick_step = max(1, len(all_bins) // 8)
        ax2.set_xticks(all_bins[::tick_step])
        ax2.set_xticklabels([f"{i*bin_size_seconds}s" for i in all_bins[::tick_step]], rotation=45)
    
    # Ensure the plot shows the full timeline
    ax2.set_xlim(-0.5, num_bins + 0.5)
    
    plt.tight_layout()
    plt.show()
    
    # Additional statistics
    print(f"\n📈 Timeline Statistics:")
    print(f"   Time bins: {num_bins} ({bin_size_seconds}s each)")
    print(f"   Peak alerts in single interval: {max(alerts_counts)} alerts")
    print(f"   Average alerts per interval: {np.mean(alerts_counts):.1f}")
    print(f"   Empty intervals: {alerts_counts.count(0)}/{num_bins} ({alerts_counts.count(0)/num_bins*100:.1f}%)")

In [ ]:
# Run the comprehensive simulation with full 75-second duration
comprehensive_results, comprehensive_alerts = run_comprehensive_alert_simulation(
    duration_seconds=75,  # Full 75 seconds for complete timeline
    samples_per_second=4
)

## Step 7: Alert Analysis and Visualization

Analyze the generated alerts and create visualizations for system performance.

In [ ]:
def run_comprehensive_alert_simulation(duration_seconds=90, samples_per_second=4):
    """
    Comprehensive alert simulation with diverse scenarios and patterns
    
    This enhanced simulation includes:
    - Multiple alert scenarios (burst patterns, gradual escalation, system health)
    - Temporal anomaly patterns (coordinated attacks, system degradation)
    - Performance-based alerts (latency spikes, accuracy drops)
    - Mixed confidence levels for realistic alert variety
    """
    
    print(f"Starting Comprehensive Multi-Scenario Alert Simulation")
    print(f"{'='*70}")
    print(f"Duration: {duration_seconds} seconds")
    print(f"High-frequency sampling: {samples_per_second} samples/second")
    print(f"Expected ~{duration_seconds * samples_per_second} total samples")
    print(f"{'='*70}")

    # Reset all systems
    alert_system.alert_history.clear()
    alert_system.alert_counts.clear()
    alert_system.last_alert_time.clear()
    
    # Temporarily reduce cooldown for more alerts
    original_cooldown = alert_system.cooldown_seconds
    alert_system.cooldown_seconds = 2  # Reduced cooldown for demo
    
    # Enhanced simulation parameters
    sample_interval = 1.0 / samples_per_second
    total_samples = int(duration_seconds * samples_per_second)
    monitor_interval = max(8, duration_seconds // 8)  # Frequent monitoring
    
    # Define alert scenarios throughout the simulation
    scenario_timeline = {
        # Early phase: Normal operation with occasional anomalies
        (0, 0.2): {
            'name': 'Normal Operations',
            'anomaly_rate': 0.15,
            'physical_weight': 0.4,
            'network_weight': 0.6,
            'burst_probability': 0.05
        },
        # Mid-early: Coordinated attack simulation
        (0.2, 0.35): {
            'name': 'Coordinated Network Attack',
            'anomaly_rate': 0.7,
            'physical_weight': 0.2,
            'network_weight': 0.8,
            'burst_probability': 0.3
        },
        # Mid: System recovery and mixed threats
        (0.35, 0.5): {
            'name': 'Mixed Threat Environment',
            'anomaly_rate': 0.45,
            'physical_weight': 0.6,
            'network_weight': 0.4,
            'burst_probability': 0.15
        },
        # Mid-late: Physical system stress test
        (0.5, 0.7): {
            'name': 'Vehicle System Stress Test',
            'anomaly_rate': 0.6,
            'physical_weight': 0.8,
            'network_weight': 0.2,
            'burst_probability': 0.25
        },
        # Final: System degradation simulation
        (0.7, 1.0): {
            'name': 'System Performance Degradation',
            'anomaly_rate': 0.3,
            'physical_weight': 0.5,
            'network_weight': 0.5,
            'burst_probability': 0.1,
            'inject_performance_issues': True
        }
    }
    
    start_time = time.time()
    last_monitor_time = start_time
    performance_degradation_active = False
    burst_mode_samples = 0
    
    print(f"Processing with dynamic scenario changes...")
    print(f"Monitor updates every {monitor_interval} seconds")
    print(f"\nSimulation starting...")
    
    try:
        for sample_num in range(total_samples):
            loop_start = time.time()
            progress_ratio = sample_num / total_samples
            
            # Determine current scenario
            current_scenario = None
            for (start_ratio, end_ratio), scenario in scenario_timeline.items():
                if start_ratio <= progress_ratio < end_ratio:
                    current_scenario = scenario
                    break
            
            if current_scenario is None:
                current_scenario = list(scenario_timeline.values())[-1]  # Use last scenario
            
            # Burst mode logic
            inject_anomaly = None
            if burst_mode_samples > 0:
                # Continue burst
                inject_anomaly = np.random.choice([1, 2], 
                    p=[current_scenario['physical_weight'], current_scenario['network_weight']])
                burst_mode_samples -= 1
            elif np.random.random() < current_scenario.get('burst_probability', 0):
                # Start new burst (3-7 consecutive anomalies)
                burst_mode_samples = np.random.randint(3, 8)
                inject_anomaly = np.random.choice([1, 2], 
                    p=[current_scenario['physical_weight'], current_scenario['network_weight']])
            elif np.random.random() < current_scenario['anomaly_rate']:
                # Regular anomaly injection
                inject_anomaly = np.random.choice([1, 2], 
                    p=[current_scenario['physical_weight'], current_scenario['network_weight']])
            
            # Get sample and run inference
            sample_data = data_streamer.get_next_sample(inject_anomaly=inject_anomaly)
            prediction_result = inference_engine.run_inference(sample_data)
            
            # Simulate performance degradation in final phase
            if current_scenario.get('inject_performance_issues', False):
                # Randomly degrade inference time
                if np.random.random() < 0.2:
                    prediction_result['inference_time_ms'] *= np.random.uniform(2.0, 4.0)
                
                # Occasionally flip predictions to simulate model degradation
                if np.random.random() < 0.12:
                    original_class = prediction_result['predicted_class']
                    alternatives = [c for c in [0, 1, 2] if c != original_class]
                    new_class = np.random.choice(alternatives)
                    prediction_result['predicted_class'] = new_class
                    prediction_result['predicted_label'] = inference_engine.class_names[new_class]
                    prediction_result['confidence'] *= np.random.uniform(0.4, 0.8)  # Lower confidence
                    prediction_result['is_correct'] = (new_class == sample_data['true_label'])
            
            # Enhanced alert triggering with performance monitoring
            should_alert = alert_system.should_trigger_alert(prediction_result)
            
            # Additional alert conditions
            if not should_alert:
                # Alert on high inference time (performance issue)
                if prediction_result['inference_time_ms'] > 50:  # Threshold for concern
                    should_alert = True
                    # Modify alert for performance issue
                    prediction_result['performance_alert'] = True
                
                # Alert on confidence drops
                elif prediction_result['confidence'] < 0.4 and prediction_result['predicted_class'] != 0:
                    should_alert = True
                    prediction_result['low_confidence_alert'] = True
            
            # Trigger alerts
            if should_alert:
                alert_data = alert_system.trigger_alert(prediction_result)
                
                # Add scenario context to alert
                alert_data['scenario'] = current_scenario['name']
                if hasattr(prediction_result, 'performance_alert'):
                    alert_data['alert_subtype'] = 'Performance Issue'
                elif hasattr(prediction_result, 'low_confidence_alert'):
                    alert_data['alert_subtype'] = 'Low Confidence'
                
                # Special handling for burst alerts
                if burst_mode_samples > 0:
                    alert_data['burst_alert'] = True
                    alert_data['remaining_burst'] = burst_mode_samples
            
            # Monitor updates with scenario info
            current_time = time.time()
            if current_time - last_monitor_time >= monitor_interval:
                metrics = monitor.log_metrics(inference_engine, alert_system, data_streamer)
                monitor.display_status(metrics)
                
                # Display current scenario
                print(f"Current Scenario: {current_scenario['name']}")
                if burst_mode_samples > 0:
                    print(f"🔥 BURST MODE ACTIVE: {burst_mode_samples} samples remaining")
                print(f"Anomaly Rate: {current_scenario['anomaly_rate']:.0%}")
                
                last_monitor_time = current_time
            
            # Maintain timing
            elapsed = time.time() - loop_start
            sleep_time = max(0, sample_interval - elapsed)
            if sleep_time > 0:
                time.sleep(sleep_time)
            
            # Progress with scenario transitions
            if (sample_num + 1) % max(1, (total_samples // 15)) == 0:
                progress = (sample_num + 1) / total_samples * 100
                elapsed_sim_time = time.time() - start_time
                print(f"Progress: {progress:.0f}% ({elapsed_sim_time:.1f}s) - {current_scenario['name']}")

    except KeyboardInterrupt:
        print(f"\n🛑 Simulation interrupted by user")
    
    finally:
        # Restore original cooldown
        alert_system.cooldown_seconds = original_cooldown
    
    # Final comprehensive statistics
    total_runtime = time.time() - start_time
    final_metrics = monitor.log_metrics(inference_engine, alert_system, data_streamer)
    
    print(f"\n🎯 Comprehensive Simulation Complete!")
    print(f"{'='*70}")
    print(f"Total runtime: {total_runtime:.2f} seconds")
    print(f"Samples processed: {final_metrics['total_predictions']}")
    print(f"Processing rate: {final_metrics['total_predictions']/total_runtime:.1f} samples/sec")
    print(f"Average inference: {final_metrics['avg_inference_time_ms']:.2f}ms")
    print(f"Recent accuracy: {final_metrics['recent_accuracy']:.1%}")
    
    # Detailed alert analysis
    alert_summary = alert_system.get_alert_summary()
    if isinstance(alert_summary, dict):
        print(f"\n📊 Comprehensive Alert Analysis:")
        print(f"   🚨 Total alerts generated: {alert_summary['total_alerts']}")
        print(f"   ⏰ Recent alerts (5 min): {alert_summary['recent_alerts']}")
        
        # Alert breakdown by type
        class_names = {0: 'Normal Operation', 1: 'Physical Anomaly', 2: 'Network Anomaly'}
        print(f"\n   Alert Distribution:")
        for alert_type, count in alert_summary['alert_counts'].items():
            percentage = (count / alert_summary['total_alerts']) * 100 if alert_summary['total_alerts'] > 0 else 0
            print(f"     {class_names.get(alert_type, f'Type {alert_type}')}: {count} alerts ({percentage:.1f}%)")
        
        # Alert rate analysis
        if alert_summary['total_alerts'] > 0:
            alert_rate = alert_summary['total_alerts'] / (total_runtime / 60)  # alerts per minute
            print(f"\n   📈 Alert Rate: {alert_rate:.1f} alerts/minute")
    
    # Save comprehensive results
    alerts_df = alert_system.save_alerts_log()
    
    # Enhanced performance plotting
    if len(monitor.metrics_history) > 1:
        monitor.plot_performance_metrics()
    
    # Alert timeline visualization
    if len(alert_system.alert_history) > 0:
        plot_alert_timeline()
    
    return final_metrics, alerts_df

print("✅ Comprehensive Alert Simulation Function Defined")
print("   Ready for multi-scenario simulation with dynamic timeline visualization")

In [ ]:
def plot_alert_timeline():
    """Create a timeline visualization of alerts with adaptive time binning and full timeline support"""
    
    if not alert_system.alert_history:
        print("No alerts to plot")
        return
    
    df_alerts = pd.DataFrame(alert_system.alert_history)
    
    # Display alert timeline info
    total_alerts = len(df_alerts)
    duration_seconds = (df_alerts['timestamp'].max() - df_alerts['timestamp'].min()).total_seconds()
    print(f"\n📊 Alert Timeline Analysis:")
    print(f"   Total alerts: {total_alerts}")
    print(f"   Simulation duration: {duration_seconds:.1f} seconds ({duration_seconds/60:.1f} minutes)")
    print(f"   Alert rate: {total_alerts/(duration_seconds/60):.1f} alerts/minute")
    
    # Create timeline plot with improved sizing for full timeline
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 12))  # Wider plot for full timeline
    
    # Alert timeline (top plot) - Fixed to show full timeline
    class_colors = {0: 'green', 1: 'red', 2: 'orange'}
    class_names = {0: 'Normal', 1: 'Physical', 2: 'Network'}
    
    # Calculate time in minutes for better readability
    for alert_type in df_alerts['type'].unique():
        type_alerts = df_alerts[df_alerts['type'] == alert_type]
        times = [(t - df_alerts['timestamp'].min()).total_seconds() / 60 for t in type_alerts['timestamp']]
        
        ax1.scatter(times, [alert_type] * len(times), 
                   c=class_colors[alert_type], label=class_names[alert_type], 
                   alpha=0.7, s=60)
    
    # Set x-axis to show full timeline
    max_time_minutes = duration_seconds / 60
    ax1.set_xlim(0, max_time_minutes)
    ax1.set_xlabel('Time (minutes)')
    ax1.set_ylabel('Alert Type')
    ax1.set_title(f'Alert Timeline - Full {duration_seconds:.1f} Second Simulation')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Set y-axis ticks to show alert type labels
    ax1.set_yticks([0, 1, 2])
    ax1.set_yticklabels(['Normal', 'Physical', 'Network'])
    
    # Alert frequency over time (bottom plot) - Improved binning with full timeline coverage
    total_duration_minutes = duration_seconds / 60
    
    if total_duration_minutes <= 2:
        # For short simulations, use 10-second bins
        bin_size_seconds = 10
        time_unit = "10-sec intervals"
    elif total_duration_minutes <= 5:
        # For medium simulations, use 15-second bins  
        bin_size_seconds = 15
        time_unit = "15-sec intervals"
    else:
        # For long simulations, use 30-second bins
        bin_size_seconds = 30
        time_unit = "30-sec intervals"
    
    # Create time bins covering the full simulation duration
    num_bins = int(np.ceil(duration_seconds / bin_size_seconds))
    df_alerts['time_bin'] = ((df_alerts['timestamp'] - df_alerts['timestamp'].min()).dt.total_seconds() / bin_size_seconds).astype(int)
    
    # Count alerts per bin
    alerts_per_bin = df_alerts.groupby('time_bin').size()
    
    # Create complete range of bins (including empty ones) for full timeline
    all_bins = range(0, num_bins + 1)
    alerts_counts = [alerts_per_bin.get(i, 0) for i in all_bins]
    
    # Create bar chart with full timeline
    ax2.bar(all_bins, alerts_counts, alpha=0.7, color='skyblue', edgecolor='navy', linewidth=0.5)
    ax2.set_xlabel(f'Time ({time_unit})')
    ax2.set_ylabel('Alerts per Interval')
    ax2.set_title(f'Alert Frequency Distribution - Full Timeline ({time_unit})')
    ax2.grid(True, alpha=0.3)
    
    # Set x-axis labels for better readability (show key time points)
    if len(all_bins) <= 20:  # Show all labels if not too crowded
        tick_step = max(1, len(all_bins) // 10)
        ax2.set_xticks(all_bins[::tick_step])
        ax2.set_xticklabels([f"{i*bin_size_seconds}s" for i in all_bins[::tick_step]], rotation=45)
    else:  # Show fewer labels for very long simulations
        tick_step = max(1, len(all_bins) // 8)
        ax2.set_xticks(all_bins[::tick_step])
        ax2.set_xticklabels([f"{i*bin_size_seconds}s" for i in all_bins[::tick_step]], rotation=45)
    
    # Ensure the plot shows the full timeline
    ax2.set_xlim(-0.5, num_bins + 0.5)
    
    plt.tight_layout()
    plt.show()
    
    # Additional statistics
    print(f"\n📈 Timeline Statistics:")
    print(f"   Time bins: {num_bins} ({bin_size_seconds}s each)")
    print(f"   Peak alerts in single interval: {max(alerts_counts)} alerts")
    print(f"   Average alerts per interval: {np.mean(alerts_counts):.1f}")
    print(f"   Empty intervals: {alerts_counts.count(0)}/{num_bins} ({alerts_counts.count(0)/num_bins*100:.1f}%)")

In [ ]:
# Run the comprehensive simulation with full 75-second duration
comprehensive_results, comprehensive_alerts = run_comprehensive_alert_simulation(
    duration_seconds=75,  # Full 75 seconds for complete timeline
    samples_per_second=4
)